In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

model =init_chat_model("groq:qwen/qwen3-32b")

In [2]:
model.invoke('hi')

AIMessage(content='<think>\nOkay, the user wrote "hi". That\'s a greeting. I should respond in a friendly and welcoming way. Maybe start with a greeting and offer help. Let me make sure the tone is positive and open. Also, check if there\'s any specific context I need to consider, but since it\'s just a simple hi, probably not. Keep it concise but inviting. Alright, something like "Hello! How can I assist you today?" Yeah, that sounds good.\n</think>\n\nHello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 9, 'total_tokens': 118, 'completion_time': 0.218739992, 'completion_tokens_details': None, 'prompt_time': 0.000379534, 'prompt_tokens_details': None, 'queue_time': 0.052230914, 'total_time': 0.219119526}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c0541-f953-7830

In [10]:
from typing_extensions import Literal
import operator
from typing import TypedDict,List,Annotated
from pydantic import BaseModel,Field
from langchain_core.messages import HumanMessage,SystemMessage

In [8]:
## schema for structured output to use in planning
class Section(BaseModel):
    name:str=Field(description="Name for this section of report")
    description:str=Field(description="Brief overview of the main topics and concepts of the section")

class Sections(BaseModel):
    sections:List[Section]=Field(
        description="Sections of the report"
    )

planner = model.with_structured_output(Sections)

In [9]:
planner

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000021D9A15CA90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021D9A2C4E90>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Sections', 'description': '', 'parameters': {'properties': {'sections': {'description': 'Sections of the report', 'items': {'properties': {'name': {'description': 'Name for this section of report', 'type': 'string'}, 'description': {'description': 'Brief overview of the main topics and concepts of the section', 'type': 'string'}}, 'required': ['name', 'description'], 'type': 'object'}, 't

### Creating workers Dynamically in Langgraph

In [12]:
from langgraph.constants import Send

class State(TypedDict):
    topic:str
    sections:list[Section]
    completed_sections:Annotated[
        list,operator.add
    ]
    final_report:str

class WorkerState(TypedDict):
    section:Section
    completed_section:Annotated[list,operator.add]

C:\Users\Siddu\AppData\Local\Temp\ipykernel_17984\1895118006.py:1: LangGraphDeprecatedSinceV10: Importing Send from langgraph.constants is deprecated. Please use 'from langgraph.types import Send' instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  from langgraph.constants import Send


In [ ]:
def orchestrator(state:State):
    """Orchestrator that generates a plan for the report"""

    report_sections = planner.invoke(
        [
            SystemMessage(content="Generate a plan for the report."),
            HumanMessage(content=f"Here is the report topic:{state['topic']}")
        ]
    )

    print("report Section:",report_sections)
    return {"sections":report_sections.sections}